In [1]:
import queue

class bt:
    def __init__(self,entropy,lvl,split_feature,gain,cls_name):
        self.split_feature = split_feature 
        self.entropy = entropy
        self.gain = gain
        self.lvl = lvl
        self.cls_name = cls_name
        self.right = None
        self.left = None



def printbt1(root,s):
    if root is None:
         return
    print(s)
    print("Level :- " ,root.lvl)
    print("Entropy :- " ,root.entropy)
    print("Split_Feature :- ",root.split_feature)
    print("Gain :- ",root.gain)
    print("Class Name :- ",root.cls_name)
    print()
    
    printbt1(root.left,"Left Node")
    printbt1(root.right,"Right Node")

In [2]:
 

from sklearn import datasets
import pandas as pd
import numpy as np
import math as ma
import sys
np.seterr(divide='ignore', invalid='ignore')
iris = datasets.load_iris()



df = pd.DataFrame(iris.data)                                #creating dataframe for iris data
bf = pd.DataFrame(iris.target)                               #creating dataframe for iris target
df.columns = iris.feature_names                             #Assigning feature names to columns



def entropy(y):                                                         #To calculate entropy
    total_elements=len(y)                           
    no_of_setosa,no_of_versicolor,no_of_virginica = count(y)            #count of different individual classes
    total=no_of_setosa+no_of_versicolor+no_of_virginica                 #total count of individulal classes
    p1=no_of_setosa/total                                               
    p2=no_of_versicolor/total
    p3=no_of_virginica/total
    return_value=0
    if p1!=0:
        return_value+=p1*ma.log(p1,2)
    if p2!=0:
        return_value+=p2*ma.log(p2,2)
    if p3!=0:
        return_value+=p3*ma.log(p3,2)
    if not return_value:
        return return_value
    return_value=-return_value                                          #entropy of current given y output data
    return return_value
    


def count(y):                                                           #to calculate count of setosa,versicolar and virginica
    setosa = np.array(y[:])
    count_setosa = (setosa==0).sum()
    
    versicolar = np.array(y[:])
    count_versi = (versicolar==1).sum()
    
    count_virgi = len(y) - (count_setosa + count_versi)
    return count_setosa,count_versi,count_virgi



def split(y1,y2,y):                                                     #to calculate the split info
    p1 = len(y1)/len(y)
    p2 = len(y2)/len(y)
    result = 0
    if p1!=0:
        result += p1*ma.log(p1,2)
    if p2!=0:
        result += p2*ma.log(p2,2)
    return -result



def feature_split(x,y,sf):                              #to calculate the best split value at which the gain ratio is maximum
    xe = np.array(x[sf])                                #creating a numpy array of values of x's sf column                                            
    temp_gain = -1                                      #a variable to max gain
    split_value = 0                                     #a variable to store best split value
    for i in range(1,len(x)):
        splitt = (xe[i-1] + xe[i])/2                    #calculating mid of two values of xe
        x_split1 = x[x[sf]>splitt]                      #splitting data of x where x[sf] is greater than splitt
        x_split2 = x[x[sf]<=splitt]                     #splitting data of x where x[sf] is less than or equal to splitt
        y_split1 = y[x[sf]>splitt]                      #splitting data of y where x[sf] is greater than splitt
        y_split2 = y[x[sf]<=splitt]                     #splitting data of y where x[sf] is less than or equal to splitt
        
        final_entropy = 0                               #variable for final entropy
        initial_entropy = entropy(y)                    #variable for initial entropy

        entropy1 = entropy(y_split1)                    #entropy of first split
        entropy2 = entropy(y_split2)                    #entropy of second split   
        final_entropy += entropy1 * len(y_split1)/len(y)                  
        final_entropy += entropy2 * len(y_split2)/len(y)                  #calculating final entropy
        info_gain = initial_entropy - final_entropy                       #calculating information gain
        split_info = split(y_split1,y_split2,y)                           #calculating split info
        gain_r = gain(info_gain,split_info)                               #calculating gain ratio
        if temp_gain < gain_r:                                            #to get maximum gain ratio and best split value
            temp_gain = gain_r
            split_value = splitt
            
    
    return temp_gain,split_value                                          #returning best gain ratio and split value of sf feature
        

def gain(info_gain,split_info):                                           #to calculate gain ratio
    
    gain_ratio = info_gain/split_info
    return gain_ratio
   

def dt(x,y,features,lvl):    
    no_of_features_left=len(features)                                     
    total_elements=len(x)
    no_of_setosa,no_of_versicolor,no_of_virginica = count(y)     
    max_cls = max(no_of_setosa,no_of_versicolor,no_of_virginica)
    if no_of_setosa == max_cls:
        cls_name = "Setosa"
    elif no_of_versicolor == max_cls:
        cls_name = "Versicolor"
    else:
        cls_name = "Virginica"
    
    if no_of_setosa==total_elements or no_of_versicolor==total_elements or no_of_virginica==total_elements or no_of_features_left==0:
        root = bt(entropy(y),lvl,"Can't Split Reached Leaf Node",0,cls_name)
        return root
    else:
        maxg = -1                                                         #variable to hold maximum gain ratio value
        split_value = 0                                                   #variable to hold best split value
        for i in features:
            temp_gain,temp_split_value = feature_split(x,y,i)             #getting maximum gain and best split value for feature i
            if maxg < temp_gain:                                          #to get the maximum gain among different features
                maxg = temp_gain                                            
                split_value = temp_split_value                            #to get the best split value among different features
                split_feature = i                                         #best feature to split on
       
        root = bt(entropy(y),lvl,split_feature,maxg,cls_name)
        x1 = x[x[split_feature]>split_value]                               #splitting data of x where x[split_feature] is greater than split_value
        y1 = y[x[split_feature]>split_value]                               #splitting data of y where x[split_feature] is greater than split_value
        x2 = x[x[split_feature]<=split_value]                              #splitting data of x where x[split_feature] is less than or equal to split_value
        y2 = y[x[split_feature]<=split_value]                              #splitting data of y where x[split_feature] is less than or equal to split_value

        root.left = dt(x1,y1,features,lvl+1)
                                                                          #using recursion to get the values
        root.right = dt(x2,y2,features,lvl+1)
       
        
        return root
root = dt(df,bf,df.columns,0)
printbt1(root,"Root Node")


Root Node
Level :-  0
Entropy :-  1.584962500721156
Split_Feature :-  petal width (cm)
Gain :-  0.9999999999999999
Class Name :-  Setosa

Left Node
Level :-  1
Entropy :-  1.0
Split_Feature :-  petal width (cm)
Gain :-  0.6933647985912662
Class Name :-  Versicolor

Left Node
Level :-  2
Entropy :-  0.15109697051711368
Split_Feature :-  petal length (cm)
Gain :-  0.2622302372762406
Class Name :-  Virginica

Left Node
Level :-  3
Entropy :-  0.0
Split_Feature :-  Can't Split Reached Leaf Node
Gain :-  0
Class Name :-  Virginica

Right Node
Level :-  3
Entropy :-  0.9182958340544896
Split_Feature :-  sepal width (cm)
Gain :-  1.0
Class Name :-  Virginica

Left Node
Level :-  4
Entropy :-  0.0
Split_Feature :-  Can't Split Reached Leaf Node
Gain :-  0
Class Name :-  Versicolor

Right Node
Level :-  4
Entropy :-  0.0
Split_Feature :-  Can't Split Reached Leaf Node
Gain :-  0
Class Name :-  Virginica

Right Node
Level :-  2
Entropy :-  0.44506485705083865
Split_Feature :-  petal length (cm)
